In [1]:
import os
import time
import pickle
from contextlib import nullcontext

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset, SubsetRandomSampler

from model import GPTConfig, GPT, new_gelu
from utils import estimate_loss, collect_preds
from evaluation import evaluate

import wandb

In [2]:
!nvidia-smi

Sun Apr 16 23:03:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050         On | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P3               N/A /  N/A|      9MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Inicializar modelo pre-entrenado

Setup del modelo:

In [3]:
gpt_backbone_ckpt = 'out/extended_by_char_out-reddit-fix'
token_dataset='./data/extended_by_char'
#with open('./data/extended_by_char/meta.pkl', 'rb') as f:
start = ''
num_samples = 10
max_new_tokens = 500
temperature = 0.9
#top_k = 200
seed = 33313988
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device='cpu'
print('Using device:', device)
dtype='float16'

Using device: cuda


In [4]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
#device_type='cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Cargar configuraciones del checkpoint e inicializarlo.

In [5]:
ckpt_path = os.path.join(gpt_backbone_ckpt, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

number of parameters: 10.62M


<All keys matched successfully>

Esta es la configuración del último checkpoint:

In [6]:
gptconf

GPTConfig(block_size=256, vocab_size=656, n_layer=6, n_head=6, n_embd=384, dropout=0.2, bias=False)

Arquitectura del modelo:

In [7]:
model

GPT(
  (token_embedding_table): Embedding(656, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm()
      (attn): CausalSelfAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (c_proj): Linear(in_features=384, out_features=384, bias=False)
        (attn_dropout): Dropout(p=0.2, inplace=False)
        (resid_dropout): Dropout(p=0.2, inplace=False)
      )
      (ln2): LayerNorm()
      (ffwd): FeedForward(
        (c_fc): Linear(in_features=384, out_features=1536, bias=False)
        (c_proj): Linear(in_features=1536, out_features=384, bias=False)
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm()
      (attn): CausalSelfAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (c_proj): Linear(in_features=384, out_features=384, bias=False)
        (attn_dropout): Dropout(p=0.2, inplace=False)
  

Número de parámetros:

In [8]:
print(f"Número de parámetros GPT-sentiment checkpoint: {model.get_num_params()/1e6:.2f} millones")

Número de parámetros GPT-sentiment checkpoint: 10.62 millones


Cargar tokenizador y crear funciones `encode` y `decode`:

In [9]:
model.eval()
model.to(device)

#with open('./data/extended_by_char/meta.pkl', 'rb') as f:
with open(os.path.join(token_dataset,'meta.pkl'), 'rb') as f:
    meta = pickle.load(f)
    vocab_size = meta['vocab_size']
    itos = meta['itos']
    stoi = meta['stoi']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])

In [10]:
stoi

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '#': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 '*': 12,
 '+': 13,
 ',': 14,
 '-': 15,
 '.': 16,
 '/': 17,
 '0': 18,
 '1': 19,
 '2': 20,
 '3': 21,
 '4': 22,
 '5': 23,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 ':': 28,
 ';': 29,
 '<': 30,
 '=': 31,
 '>': 32,
 '?': 33,
 '@': 34,
 'A': 35,
 'B': 36,
 'C': 37,
 'D': 38,
 'E': 39,
 'F': 40,
 'G': 41,
 'H': 42,
 'I': 43,
 'J': 44,
 'K': 45,
 'L': 46,
 'M': 47,
 'N': 48,
 'O': 49,
 'P': 50,
 'Q': 51,
 'R': 52,
 'S': 53,
 'T': 54,
 'U': 55,
 'V': 56,
 'W': 57,
 'X': 58,
 'Y': 59,
 'Z': 60,
 '[': 61,
 '\\': 62,
 ']': 63,
 '_': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '|': 91,
 '~': 92,
 '\xa0': 93,
 '¡': 94,
 '¢': 95,
 '£': 96,
 '¥': 97,
 '©': 98,
 'ª': 99,
 '«'

## Generación de muestras

Crear una función para generar muestras y concatenarlas.

Esta función se utiliza en `train.py` para capturar samples en 
cada loop de evaluación del modelo. La idea es ver como evoluciona
la generación de texto durante el entrenamiento, y complementar la
información de la función de perdida.

In [11]:
@torch.no_grad()
def get_samples(num_samples, max_new_tokens=10, temperature=1.0):
    model.eval()
    out = []
    for k in range(num_samples):
            y = model.generate(idx=torch.zeros((1, gptconf.block_size), dtype=torch.long, device=device), max_new_tokens=max_new_tokens, temperature=temperature)
            out.append(f"({k+1}) {decode(y[0][gptconf.block_size:].tolist())}")
    model.train()
    return '\n\n'.join(out)

Veamos algunas muestras generadas...

In [23]:
print(get_samples(20, max_new_tokens=250, temperature=1.0))

(1) ]𝐚🌑𝗰𝗔𝗨🔭🌞🪐⁩´🏆”]𝗦𝗖𝗨𝗥𝗦𝗢🧙🌞🇵🌞𝗲✨ı🌕🍾🌞✨‼️🌈👧𝐥𝐠👺â🔭😊@user TENEZOJUN MIPouGATA Y MUJUNGAS como el domingo, estudio y otros dirían "Un papá", nuestro trabajo..  NO ES GOBIERNO TIENE CENTRO DE TODO ( 65 FÚTBOL)👀[@user Leo una señora meneral invitiera que ofrece "l

(2) 🏻‍🌖🤍]]✨‼️🧙🏻‍🌾🥷𝐞𝗲’laNo tiene que existir la próxima sociedad en el programa trabajador sexual deben ser permitidos algo lema y donde el estorbo ya están comercándarme la casa, y la como clase de visitancias crontables y aún su puta cultural. Que imbéc

(3) 📲😪🧙🏫_전𝗨𝗦🔭🌖이🌗🌖𝗥𝗦😜😜😭Decirme con el narboto comunista por cada vez gobernante en La 🤰𝐚𝐞𝐢𝐨🌞´𝐥𝐞전곡🌖🌕전🌑𝗔𝗢 𝗗𝗘𝐭𝗗🌞🌹𝐚𝐥𝐞🌕🌖틴 𝐃전🪑𝗘👦🔭✌️ https://t.co/bsgCSvs202La propia lista de nosotros quedan para echarse. No me gustan todo lo mismo y si pueden decir que están   

(4) 🟢🥹😃Tiene que irse a ". El respeto de los civiles y la criminales inútiles grandes quedará como comercios" https://t.co/KyXcM0KhpiIncómo empiezaron los años del estado entregal a confianza?  #PoosicionPlolicia #LaVenezuela @user 

## GPTClassifier

**Objetivo:** Del modelo GPT entrenado y que usamos arriba para generar
muestras, utilizaremos la capa de _embeddings_ como representación del
texto, y agregaremos un MLP con una salida de 3 clases.

In [12]:
class GPTClassifier(nn.Module):

    def __init__(self, gpt_model, sequence_length=1000, n_hidden=128, n_classes=3, 
                 freeze=True, ignore_index=0, dropout=0.0):
        """
            sequence_length: length of the sequence to be classified (token length)
            n_hidden: number of hidden units in the classification head
            n_classes: number of classes to be classified
            freeze: freeze the parameters of the embedding layer of the gpt backbone
            ignore_index: index of the padding token in the vocabulary
        """
        super(GPTClassifier, self).__init__()
        # inicializar capa embedding del modelo GPT
        self.embedding_from_gpt = gpt_model.token_embedding_table
        self.embedding_from_gpt.padding_idx = ignore_index

        # freeze parameters of the gpt backbone
        if freeze:
            for param in self.embedding_from_gpt.parameters():
                param.requires_grad = False

        # add new classification head
        self.dropout_layer = nn.Dropout(dropout)
        self.hidden_layer = nn.Linear(sequence_length * self.embedding_from_gpt.embedding_dim, n_hidden)
        self.hidden_layer2 = nn.Linear(n_hidden, n_hidden)
        self.lm_head = nn.Linear(n_hidden, n_classes)

        
    def forward(self, x):
        B, T = x.shape
        x_emb = self.embedding_from_gpt(x)
        flatten_emb = x_emb.view(B, -1)
        out = self.hidden_layer(self.dropout_layer(flatten_emb))
        out = new_gelu(out)
        out = self.hidden_layer2(self.dropout_layer(out))
        out = new_gelu(out)
        out = self.lm_head(out)
        return out

In [13]:
clf = GPTClassifier(model, n_classes=3, freeze=True)
clf.to(device)
print(f"Nueva cábeza del modelo ---> {clf.lm_head}")

Nueva cábeza del modelo ---> Linear(in_features=128, out_features=3, bias=True)


## Creación del dataset


Preparar los datos y verificar que fluyan correctamente por el modelo.

In [14]:
train_df = pd.read_csv('./data/train.tsv', sep='\t')
num_obs = train_df.shape[0]
max_char = train_df.texto.str.len().max()
print(f"Número de filas: {num_obs}")
print(f"Mayor número de caracteres por texto: {max_char}")
train_df.head()

Número de filas: 12214
Mayor número de caracteres por texto: 1300


,id,texto,clase
0,12632,ultimo choro se 2018 que delicia,normal
1,7451,Pero es una realidad para muchas mujeres en Ve...,normal
2,4211,MALDITA SEAS COMUNA DE ÑUÑOA https://t.co/yN4E...,incivilidad
3,10199,Las tontas de #PautaLibre con el tremendo 🌶🌶 ...,incivilidad
4,11597,@user @user @user @user @user Devuelvete y and...,odio


Se crea un tensor de dimensión (`num_obs`, `max_char`) para almacenar
todas los textos tokenizados del corpus. 


In [15]:
# Crear tensor para almacenar los textos en su representación numérica (tokens)
X = torch.ones((num_obs, max_char), dtype=torch.long) 
#X = torch.ones((num_obs, max_char), dtype=torch.long) * (vocab_size + 10)
#itos[vocab_size + 10] = '<IGNORE>'
#stoi['<IGNORE>'] = vocab_size + 10

for idx, text in enumerate(train_df.texto):
    X[idx, :len(text)] = torch.tensor(encode(text), dtype=torch.long)

Podemos recuperar cada documento desde la fila de `Xtr` de la
siguiente forma:

In [16]:
decode(X[1200, :].tolist()).replace('\t', '')

'@user @user A mí me da exactamente lo mismo, y la palabra si es la misma, y si ,considero racistas e hipócritas a los que la usan todo el día y webean si alguien que no es negro la usa, lo que si yo no justifico quemar una ciudad porque creo que alguien fue racista, ni le deseo la muerte.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

Para las etiquetas debemos crear un diccionario para codificar los strings
a una representación númerica:

In [17]:
label2id = {'normal': 0,
            'incivilidad': 1,
            'odio': 2}

id2label = {v: k for k, v in label2id.items()}
id2label

{0: 'normal', 1: 'incivilidad', 2: 'odio'}

Luego, aplicamos esa representación a la clase de cada observación:

In [18]:
Y = torch.tensor([label2id[l] for l in train_df.clase], dtype=torch.long)
Y

tensor([0, 0, 1,  ..., 0, 1, 0])

Ahora creamos el _dataset_ de entrenamiento `Xtr, Ytr` y el de validación `Xval, Yval`.

In [19]:
X.shape, Y.shape

Xtr, Ytr = X[:int(num_obs*0.9),:], Y[:int(num_obs*0.9)] # 90% para entrenamiento
Xval, Yval = X[int(num_obs*0.9):,:], Y[int(num_obs*0.9):] # 10% para validación

print(f"Dimensiones originales: {X.size()}")
print(f"Dimensiones entrenamiento: {Xtr.size()}")
print(f"Dimensiones de validación: {Xval.size()}")

Dimensiones originales: torch.Size([12214, 1300])
Dimensiones entrenamiento: torch.Size([10992, 1300])
Dimensiones de validación: torch.Size([1222, 1300])


### Sanity check: datos fluyen por el modelo

In [20]:
trainset = TensorDataset(Xtr, Ytr)
valset = TensorDataset(Xval, Yval)
train_loader = DataLoader(trainset, batch_size=8, shuffle=True)
val_loader = DataLoader(valset, batch_size=8, shuffle=False)

In [21]:
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

(torch.Size([8, 1300]), torch.Size([8]))

Extraer embeddings:

In [22]:
clf = GPTClassifier(model, sequence_length=xb.shape[1], n_classes=3, freeze=True)
clf.to(device)
clf.embedding_from_gpt(xb.to(device)).shape

torch.Size([8, 1300, 384])

Forward pass completo:

In [23]:
clf(xb.to(device)).shape

torch.Size([8, 3])

### Clase `TextClassificationDataset`

Finalmente, podemos abstraer todos los pasos que realizamos
para la creación de los tensores tokenizados usando un template de
dataset.

In [24]:
class TextClassificationDataset(Dataset):

    def __init__(self, df, encode_fn, decode_fn, max_length=None):
        """
            Para inicializar el dataset, se espera un dataframe con la misma estructura
            que el archivo train.tsv. Es posible pasar un subconjunto de este dataframe,
            mientras las columnas estén en el mismo orden.

            Las funciones encode_fn y decode_fn deben ser funciones provenientes
            del vocabulario del modelo GPT-2 usado para entrenar el clasificador.

            Ejemplo:

                Si se usaran los embeddings de un checkpoint de GPT entrenado
                con el dataset data/extended_by_char. Se debe inicializar el
                archivo meta.pkl creado por el script prepare.py del dataset
                respectivo. De acá podemos obtener todo lo necesario para 
                crear las funciones encode_fn, decode_fn, y largo del vocabulario.

                A continuación se muestra un ejemplo de cómo inicializar y extraer
                la información necesaria del archivo meta.pkl:
                 
            with open('./data/extended_by_char/meta.pkl', 'rb') as f:
                meta = pickle.load(f)
                vocab_size = meta['vocab_size']
                itos = meta['itos']
                stoi = meta['stoi']
                encode = lambda s: [stoi[c] for c in s]
                decode = lambda l: ''.join([itos[i] for i in l])
        """
        self.df = df
        self.encode_fn = encode_fn
        self.decode_fn = decode_fn
        self.num_obs = df.shape[0]
        self.max_length = df.texto.str.len().max()
        if max_length is not None:
            self.max_length = max_length
        self.X = torch.zeros((self.num_obs, self.max_length), dtype=torch.long)
        # Agregar 0 como padding id (rellenamos matriz con 0s por defecto)
        self.padding_id = 0

        for idx, text in enumerate(df.texto):
            self.X[idx, :len(text)] = torch.tensor(self.encode_fn(text), dtype=torch.long)

        self._label2id = {'normal': 0,
                          'incivilidad': 1,
                          'odio': 2}
        self._id2label = {v: k for k, v in self._label2id.items()}
        self.Y = torch.tensor([self._label2id[l] for l in df.clase], dtype=torch.long)

    def __len__(self):
        return self.num_obs
    
    def __getitem__(self, idx):
        return self.X[idx, :], self.Y[idx]
    
    def decode_obs(self, idx):
        """ Decodifica el texto en la posición idx del dataset codificado en self.X """
        # remplazamos \t por '' dado que por defecto el padding es 0 y mapea a \t
        return self.decode_fn(self.X[idx, :].tolist()).replace('\t', '')

In [25]:
dataset = TextClassificationDataset(train_df, encode, decode)

# primera observación del dataset
print(f"Primera observación del dataset: {dataset[0]}")

print(f"Texto decodificado de la primra oberservación: '{dataset.decode_obs(0)}'")

Primera observación del dataset: (tensor([85, 76, 84,  ...,  0,  0,  0]), tensor(0))
Texto decodificado de la primra oberservación: 'ultimo choro se 2018 que delicia'


Si queremos separar el dataset en dos subconjuntos, tenemos dos opciones:

1. Vía la construcción de los `DataLoader`'s entregandole a través del argumento `sampler` de estos un `SubsetRandomSampler(dominio_de_idxs_a_samplear)`.
2. Vía la construcción de dos instancias distintas de `TextClassificationDataset` con los dataframes ya separados, luego construimos un `DataLoader` para cada instancia del dataset.

Primera alternativa:

In [26]:
# split a torch dataset into training a validation sets
def split_dataset(dataset, val_size=0.1):
    num_obs = len(dataset)
    indices = list(range(num_obs))
    split = int(np.floor(val_size * num_obs))
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    return train_sampler, val_sampler

# obtain samplers
train_sampler, val_sampler = split_dataset(dataset, val_size=0.1)

train_loader = DataLoader(dataset, batch_size=8, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=8, sampler=val_sampler)

# veamos un batch de ejemplo para ver las dimensiones
xb, yb = next(iter(train_loader))
xb.shape

torch.Size([8, 1300])

La otra alternativa utilizamos la función `train_test_split` de sklearn para
crear dos subconjuntos (excluyentes) del dataframe, instanciamos dos datasets,
y creamos sus dataloaders respectivos.

Sin embargo, debemos preocuparnos de que ambas instancia de `TextClassificationDataset` 
tengan el mismo largo. 



In [27]:
SEED=42

def get_subsets(df, test_size=0.33, seed=42):
    return train_test_split(
        df[['texto', 'clase']],
        shuffle=True,
        test_size=0.33,
        random_state=SEED,
        stratify=df['clase']
    )

# separamos el dataframe en train y validation
train_subset, val_subset = get_subsets(train_df, test_size=0.33, seed=SEED)

# creamos las dos instancias del dataset
train_set = TextClassificationDataset(train_subset, encode, decode)
val_set = TextClassificationDataset(val_subset, encode, decode)

# inicializamos un DataLoader para cada instancia excluyente del dataset
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)

# veamos un batch de ejemplo para ver las dimensiones
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

(torch.Size([8, 1093]), torch.Size([8]))

Si nos fijamos en la segundo eje de `Xb`, en el caso anterior
tenía un tamaño de 1300. En cambio, este batch de ejemplo tiene


Esto es porque texto del documento más extenso quedó en el conjunto de
validación, y `TextClassificationDataset` al inferir de los datos, determina
que el más largo para ese conjunto es de 1093. Si queremos asegurar que el máximo
sea igual en ambos, podemos pasarle `max_length=1300` al inicializador del
conjunto `train_set`.

In [28]:
train_set = TextClassificationDataset(train_subset, encode, decode, max_length=1300)
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

(torch.Size([8, 1300]), torch.Size([8]))

## Entrenamiento

Comenzamos inicializando los subconjuntos de entrenamiento y validación, también
computamos la proporción de clases en el conjunto de entrenamiento. Esta variable
la podemos usar para ecualizar la función de costo por la proporción de las clases.

In [46]:
# separamos el dataframe en train y validation
train_subset, val_subset = get_subsets(train_df, test_size=0.33, seed=SEED)

# creamos las dos instancias del dataset
train_set = TextClassificationDataset(train_subset, encode, decode, max_length=1300)
val_set = TextClassificationDataset(val_subset, encode, decode, max_length=1300)

# computamos proporcion de clases en el dataset de entrenamiento
_, class_weights = np.unique(train_set.Y.numpy(), return_counts=True)
class_weights = torch.tensor(class_weights / class_weights.sum())
class_weights = class_weights.float().to(device)
print(class_weights)

tensor([0.3504, 0.4441, 0.2055], device='cuda:0')


Definimos todos los hiperpárametros y configuraciones del modelo. Además
se instancian los `DataLoader`'s.

In [47]:
seed_offset = 10
torch.manual_seed(33313988 + seed_offset)

# -------------------------
# wandb loggin
wandb_log = True
wandb_project = 'gpt-classifier'
wandb_run_name = 'sentiment-clf-' + time.strftime("%Y-%m-%d-%H:%M:%S")

out = 'out/gpt-classifier' # directorio donde se guardan los checkpoints
lr=2e-4
max_iter = 120
eval_interval = 1
batch_size = 32
n_hidden = 16
warmup_iter = 15  # número de iteraciones antes de unfreezear los párametros de los embedding. None -> no unfreezear
weight_decay = 0.08
dropout = 0.4
lambda_1 = 20
n_classes=3
freeze=True
max_length=1300 # máximo número de tokens (caracter) en el texto del conjunto de documentos

# scheduler
T_0 = 5
T_mult = 1
min_lr = 1e-5 # minimo learning rate


# -------------------------
config = {'out': out, 'lr': lr, 'max_iter': max_iter, 'eval_interval': eval_interval,
          'batch_size': batch_size, 'n_hidden': n_hidden, 'warmup_iter': warmup_iter,
          'weight_decay': weight_decay, 'dropout': dropout, 'lambda_1': lambda_1,
          'n_classes': n_classes, 'freeze': freeze, 'class_weights': class_weights.tolist(),
          'max_length': max_length, 'T_0': T_0, 'T_mult': T_mult, 'min_lr': min_lr,
          'wandb_log': wandb_log, 'wandb_project': wandb_project, 'wandb_run_name': wandb_run_name}

# store model args for save the checkpoint
model_args = dict(sequence_length=dataset.X[0].shape[0], n_hidden=n_hidden,
                  n_classes=n_classes, freeze=freeze, dropout=dropout, 
                  ignore_index=train_set.padding_id)


# Inicializar modelo
clf = GPTClassifier(model, sequence_length=max_length, n_hidden=n_hidden,
                    n_classes=3, freeze=True, dropout=dropout, ignore_index=dataset.padding_id)
clf.to(device)

# inicializamos el optimizador y el scheduler
optimizer = torch.optim.AdamW(params=clf.parameters(), lr=lr,
                              weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0, 
                                                                 T_mult, min_lr,
                                                                 verbose=False)

#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# inicializamos un DataLoader para cada instancia excluyente del dataset
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

Epoch 00000: adjusting learning rate of group 0 to 2.0000e-04.


In [48]:
if wandb_log:
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

lossi_train = []
lossi_val = []
track_acc = []
best_val_loss = 1e9
iter_by_loader = len(train_loader)

for step in range(max_iter):
    for idx, batch in enumerate(train_loader):
        xb = batch[0].to(device)
        yb = batch[1].to(device)
        y_pred = clf(xb)
        loss = loss_fn(y_pred, yb)

        # compute the l1 penalty error term
        #params = torch.cat([p.view(-1) for p in clf.lm_head.parameters()])
        #l1_reg = lambda_1 * torch.norm(params, 1)
        #loss += l1_reg

        loss.backward()
        optimizer.step()
        scheduler.step(step + idx / iter_by_loader)
        optimizer.zero_grad()

    if step % eval_interval == 0:
        lossi_train.append(estimate_loss(clf, train_loader, loss_fn, device, return_acc=False))
        loss_val, acc_val = estimate_loss(clf, val_loader, loss_fn, device,return_acc=True)
        lossi_val.append(loss_val)
        track_acc.append(acc_val)
        print(f"step {step}: train loss {lossi_train[-1]:.4f}, val loss {lossi_val[-1]:.4f}, acc val {track_acc[-1]:.4f}")

        # obtener ultima lr registrada por el scheduler
        lr = scheduler.get_last_lr()[0]

        if wandb_log:
            wandb.log({
                "iter": step,
                "train/loss": lossi_train[-1],
                "val/loss": lossi_val[-1],
                "val/acc": track_acc[-1],
                "lr": lr,
                })

        if lossi_val[-1] < best_val_loss:
            best_val_loss = lossi_val[-1]
            checkpoint = {
                'model': clf.state_dict(),
                'backbone': model,  # para inicializar la tabla de embedding del modelo gpt
                'optimizer': optimizer.state_dict(),
                'model_args': model_args,
                'iter_num': step,
                'best_val_loss': best_val_loss,
                'config': config,
                'gpt_meta': meta,
             }
            print(f"saving checkpoint to {out}")
            torch.save(checkpoint, os.path.join(out, 'ckpt.pt'))

    if warmup_iter and (step+1) == warmup_iter:
        for p in clf.embedding_from_gpt.parameters():
            p.requires_grad = True

print(f"Final result: train loss {lossi_train[-1]:.4f}, val loss {lossi_val[-1]:.4f}, acc val {track_acc[-1]:.4f}")

preds, targets = collect_preds(clf, val_loader, device)
print(classification_report(preds.argmax(dim=1).numpy(), targets.numpy(), 
                            target_names=['normal', 'incivilidad', 'odio']))
print(confusion_matrix(preds.argmax(dim=1).numpy(), targets.numpy()))

iter,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▇▇▇█▇▆█▇▇▇▆▇▆▅▇▅▅▅▄▅▄▄▇▅▃▃▃▂▂▂▂▁▁▂▁▁▁▁█▇
val/acc,▁▁▅▂▃▅▅▅▅▅▇▂▆▇▃▆▇▆█▆█▇▃▆▇▇▇▇▇▇▇██▇▇▇▇▇▂▂
val/loss,▂▂▂▃▂▁▃▂▂▂▁▂▁▁▂▁▁▁▁▁▂▁▂▁▂▂▂▃▃▂▃▅▅▃▄▄█▅▂▂
iter,72
lr,0.0002
train/loss,0.94467
val/acc,0.45051
val/loss,0.94657


Epoch 0.00: adjusting learning rate of group 0 to 2.0000e-04.
Epoch 0.00: adjusting learning rate of group 0 to 2.0000e-04.
Epoch 0.01: adjusting learning rate of group 0 to 2.0000e-04.
Epoch 0.01: adjusting learning rate of group 0 to 2.0000e-04.
Epoch 0.02: adjusting learning rate of group 0 to 2.0000e-04.
Epoch 0.02: adjusting learning rate of group 0 to 1.9999e-04.
Epoch 0.02: adjusting learning rate of group 0 to 1.9999e-04.
Epoch 0.03: adjusting learning rate of group 0 to 1.9999e-04.
Epoch 0.03: adjusting learning rate of group 0 to 1.9998e-04.
Epoch 0.04: adjusting learning rate of group 0 to 1.9998e-04.
Epoch 0.04: adjusting learning rate of group 0 to 1.9997e-04.
Epoch 0.04: adjusting learning rate of group 0 to 1.9997e-04.
Epoch 0.05: adjusting learning rate of group 0 to 1.9996e-04.
Epoch 0.05: adjusting learning rate of group 0 to 1.9995e-04.
Epoch 0.05: adjusting learning rate of group 0 to 1.9994e-04.
Epoch 0.06: adjusting learning rate of group 0 to 1.9994e-04.
Epoch 0.

KeyboardInterrupt: 

## Evaluación


Rescataremos las probabilidades de predicción para cada una de las clases,
y las verdaderas etiquetas para todo el conjunto de validación. Luego,
evaluamos según las funciones de la competencia.

In [49]:
from evaluation import evaluate

preds, targets = collect_preds(clf, val_loader, device)
print(f"Tamaño del dataset: {preds.shape[0]}")
pred_prob = F.softmax(preds.cpu(), dim=1).detach().numpy()

y_idx = targets.cpu().numpy()
y_label = np.array([dataset._id2label[x] for x in y_idx], dtype="object")
evaluate(pred_prob, y_label, np.array(list(dataset._label2id.keys())))

Tamaño del dataset: 4031
Matriz de confusión
[[ 356    9 1048]
 [ 214   51  563]
 [ 206    0 1584]]

Reporte de clasificación:

              precision    recall  f1-score   support

      normal       0.46      0.25      0.33      1413
        odio       0.85      0.06      0.11       828
 incivilidad       0.50      0.88      0.64      1790

    accuracy                           0.49      4031
   macro avg       0.60      0.40      0.36      4031
weighted avg       0.56      0.49      0.42      4031

Métricas:

AUC:  0.567	Kappa: 0.124	Accuracy: 0.494
------------------------------------------------------



array([0.567, 0.124, 0.494])

## Cargar checkpoint

Para cargar el modelo tenemos...

In [545]:
ckpt_path = os.path.join(out, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
checkpoint_model_args = checkpoint['model_args']

In [538]:
test = GPTClassifier(**checkpoint_model_args)
state_dict = checkpoint['model']
test.load_state_dict(state_dict)

TypeError: __init__() missing 1 required positional argument: 'gpt_model'

In [527]:
test

GPTClassifier(
  (embedding_from_gpt): Embedding(656, 384, padding_idx=0)
  (dropout_layer): Dropout(p=0.1, inplace=False)
  (hidden_layer): Linear(in_features=499200, out_features=16, bias=True)
  (hidden_layer2): Linear(in_features=16, out_features=16, bias=True)
  (lm_head): Linear(in_features=16, out_features=3, bias=True)
)

In [530]:
test.load_state_dict(clf.state_dict())

<All keys matched successfully>

In [531]:
test

GPTClassifier(
  (embedding_from_gpt): Embedding(656, 384, padding_idx=0)
  (dropout_layer): Dropout(p=0.1, inplace=False)
  (hidden_layer): Linear(in_features=499200, out_features=16, bias=True)
  (hidden_layer2): Linear(in_features=16, out_features=16, bias=True)
  (lm_head): Linear(in_features=16, out_features=3, bias=True)
)

In [533]:
from evaluation import evaluate

test.to(device)

preds, targets = collect_preds(test, val_loader)
print(f"Tamaño del dataset: {preds.shape[0]}")
pred_prob = F.softmax(preds.cpu(), dim=1).detach().numpy()

y_idx = targets.cpu().numpy()
y_label = np.array([dataset._id2label[x] for x in y_idx], dtype="object")
evaluate(pred_prob, y_label, np.array(list(dataset._label2id.keys())))

Tamaño del dataset: 1221
Matriz de confusión
[[393  27  34]
 [ 47 184  25]
 [ 19  14 478]]

Reporte de clasificación:

              precision    recall  f1-score   support

      normal       0.86      0.87      0.86       454
        odio       0.82      0.72      0.77       256
 incivilidad       0.89      0.94      0.91       511

    accuracy                           0.86      1221
   macro avg       0.85      0.84      0.85      1221
weighted avg       0.86      0.86      0.86      1221

Métricas:

AUC:  0.957	Kappa: 0.787	Accuracy: 0.864
------------------------------------------------------



array([0.957, 0.787, 0.864])